# Pipeline Verification (Keerthi)

This notebook verifies the implementation of the model loading, LoRA configuration, and training loop logic. 
It uses a small model and dummy data to ensure the code runs without errors on the local machine.

In [ ]:
import sys
import os

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..')))

from src.model import get_model_and_tokenizer, get_lora_config, get_peft_model_wrapper
from src.evaluate_metrics import compute_metrics, evaluate_model
import torch

## 1. Test Model Loading (CPU/Mac Friendly)
We use `gpt2` as a tiny placeholder to test the logic without downloading 7GB weights.

In [ ]:
# Disable quantization for local Mac testing
model_name = "gpt2" 
model, tokenizer = get_model_and_tokenizer(model_name, use_quantization=False)

print("Model loaded successfully:", type(model))

## 2. Test LoRA Configuration
Apply the LoRA adapter to the model.

In [ ]:
# Note: GPT2 uses 'c_attn' instead of 'q_proj', 'v_proj' etc.
# For this test, we might need to adjust target_modules temporarily or just check if it initializes.
# In the real script, we use Mistral/Llama target modules.

try:
    peft_config = get_lora_config(r=8, alpha=16)
    # Override target modules for GPT2 test
    peft_config.target_modules = ["c_attn"]
    
    lora_model = get_peft_model_wrapper(model, peft_config)
    print("LoRA adapter applied successfully.")
except Exception as e:
    print(f"LoRA Error (expected if modules don't match): {e}")

## 3. Test Evaluation Metrics
Run a dummy evaluation.

In [ ]:
dummy_dataset = [
    {"prompt": "History of Rome:", "completion": "Rome was founded in 753 BC."},
    {"prompt": "Who was Napoleon?", "completion": "He was a French emperor."}
]

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

metrics = evaluate_model(model, tokenizer, dummy_dataset, max_new_tokens=10)
print("Evaluation Metrics:", metrics)